In [1]:
import random
import tensorflow as tf
import numpy as np
import os
from scipy import ndimage
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
def recover_model(meta_filename):
    return tf.train.import_meta_graph(meta_filename)

In [5]:
meta_filename = 'model.ckpt-3425.meta'
checkpoint = 'model.ckpt-3425'

other_meta_filename = 'model.ckpt-345.meta'
other_checkpoint = 'model.ckpt-345'

new_meta_filename = 'sample_model/model.ckpt-0.meta'
new_checkpoint = 'sample_model/model.ckpt-0'

In [6]:
graph = tf.Graph()
with graph.as_default():
    #saver = tf.train.Saver(tf.global_variables())
    saver = recover_model(new_meta_filename)
    #saver.restore(sess, index_filename)
    
    global_step = tf.get_collection("global_step")[0]
    loss = tf.get_collection("loss")[0]
    predictions = tf.get_collection("predictions")[0]
    labels = tf.get_collection("labels")[0]
    train_op = tf.get_collection("train_op")[0]
    init_op = tf.global_variables_initializer()

sess = tf.Session(graph=graph)
saver.restore(sess, new_checkpoint)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from sample_model/model.ckpt-0


In [7]:
other_graph = tf.Graph()
with other_graph.as_default():
    other_saver = recover_model(other_meta_filename)
    
other_sess = tf.Session(graph=other_graph)
other_saver.restore(other_sess, other_checkpoint)

INFO:tensorflow:Restoring parameters from model.ckpt-345


In [8]:
transferable_variables = [var for var in graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES) if '/rnn/' in var.name]
#other_transferable_variables = [var for var in other_graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES) if '/rnn/' in var.name]

In [9]:
# sess.run(graph.get_operation_by_name('tower/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/read').values())[0][0]
# other_sess.run(other_graph.get_operation_by_name('tower/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/read').values())[0][0]
graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'tower/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(1152, 4096) dtype=float32_ref>,
 <tf.Variable 'tower/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(4096,) dtype=float32_ref>,
 <tf.Variable 'tower/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(2048, 4096) dtype=float32_ref>,
 <tf.Variable 'tower/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(4096,) dtype=float32_ref>,
 <tf.Variable 'tower/gates/weights:0' shape=(1024, 60) dtype=float32_ref>,
 <tf.Variable 'tower/experts/weights:0' shape=(1024, 40) dtype=float32_ref>,
 <tf.Variable 'tower/experts/biases:0' shape=(40,) dtype=float32_ref>]

In [8]:
# target_ts = other_graph.get_tensor_by_name('tower/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0')
# replacement_ts = graph.get_tensor_by_name('tower/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0')
graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'tower/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(1152, 4096) dtype=float32_ref>,
 <tf.Variable 'tower/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(4096,) dtype=float32_ref>,
 <tf.Variable 'tower/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(2048, 4096) dtype=float32_ref>,
 <tf.Variable 'tower/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(4096,) dtype=float32_ref>,
 <tf.Variable 'tower/gates/weights:0' shape=(1024, 60) dtype=float32_ref>,
 <tf.Variable 'tower/experts/weights:0' shape=(1024, 40) dtype=float32_ref>,
 <tf.Variable 'tower/experts/biases:0' shape=(40,) dtype=float32_ref>]

In [9]:
sess.close()
other_sess.close()

In [10]:
graph.add_to_collections('TRANSFERABLE_VARIABLES', transferable_variables)
other_graph.add_to_collections('TRANSFERABLE_VARIABLES', other_transferable_variables)

NameError: name 'other_transferable_variables' is not defined

In [13]:
transferable_weights = sess.run(graph.get_collection('TRANSFERABLE_VARIABLES'))

In [26]:
for variable, weight in zip(other_graph.get_collection('TRANSFERABLE_VARIABLES')[0],transferable_weights[0]):
    #variable.load(weight, other_sess)
    tf.assign(variable, weight)

IndexError: list index out of range

In [25]:
transferable_weights[0][0][0]

array([ 0.20338826,  0.17654507, -0.5066063 , ...,  0.02780853,
        0.5538771 , -0.01255624], dtype=float32)

In [27]:
other_sess.run(other_graph.get_collection('TRANSFERABLE_VARIABLES'))[0][0][0]

array([ 0.20338826,  0.17654507, -0.5066063 , ...,  0.02780853,
        0.5538771 , -0.01255624], dtype=float32)

In [43]:
graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'tower/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(1152, 4096) dtype=float32_ref>,
 <tf.Variable 'tower/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(4096,) dtype=float32_ref>,
 <tf.Variable 'tower/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(2048, 4096) dtype=float32_ref>,
 <tf.Variable 'tower/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(4096,) dtype=float32_ref>,
 <tf.Variable 'tower/gates/weights:0' shape=(1024, 1581) dtype=float32_ref>,
 <tf.Variable 'tower/experts/weights:0' shape=(1024, 1054) dtype=float32_ref>,
 <tf.Variable 'tower/experts/biases:0' shape=(1054,) dtype=float32_ref>]

In [49]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [50]:
!python transfer2.py --frame_features --model=LstmModel --feature_names=audio_embedding --feature_sizes=128 --train_data_pattern=tfrecord/*.tfrecord --train_dir model_new/dir --start_new_model --base_learning_rate=0.001 --num_epochs=2

INFO:tensorflow:/job:master/task:0: Tensorflow version: 1.10.1.
INFO:tensorflow:/job:master/task:0: Removing existing train directory.
INFO:tensorflow:/job:master/task:0: Flag 'start_new_model' is set. Building a new model.
2018-10-23 16:10:32.625755: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-10-23 16:10:32.646521: E tensorflow/stream_executor/cuda/cuda_driver.cc:397] failed call to cuInit: CUDA_ERROR_NO_DEVICE
2018-10-23 16:10:32.646548: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] retrieving CUDA diagnostic information for host: pct-ds1
2018-10-23 16:10:32.646555: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:170] hostname: pct-ds1
2018-10-23 16:10:32.646577: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:194] libcuda reported version is: 396.51.0
2018-10-23 16:10:32.646600: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:198] kernel reported

In [5]:
!python train.py --frame_features --model=LstmModel --feature_names=audio_embedding --feature_sizes=128 --train_data_pattern=tfrecord/*.tfrecord --train_dir model_new/dir --start_new_model --base_learning_rate=0.001 --num_epochs=20

INFO:tensorflow:/job:master/task:0: Tensorflow version: 1.11.0.
INFO:tensorflow:/job:master/task:0: Removing existing train directory.
INFO:tensorflow:/job:master/task:0: Flag 'start_new_model' is set. Building a new model.
2018-10-22 17:36:19.236374: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-10-22 17:36:19.373376: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1411] Found device 0 with properties: 
name: TITAN Xp major: 6 minor: 1 memoryClockRate(GHz): 1.582
pciBusID: 0000:09:00.0
totalMemory: 11.91GiB freeMemory: 436.06MiB
2018-10-22 17:36:19.373405: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0
2018-10-22 17:36:19.590564: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-10-22 17:36:19.590601: I tensorflow/core/common_runtime/gpu/gpu_device.cc:977]      0 
2

In [43]:
!python3

Python 3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
KeyboardInterrupt
>>> 

In [48]:
import sys
sys.version

'3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 17:14:51) \n[GCC 7.2.0]'